[Haar Face Detection](https://docs.opencv.org/4.x/d2/d99/tutorial_js_face_detection.html)

In [1]:
import cv2

In [2]:
img = cv2.imread('1.jpg')
gray_img = cv2.cvtColor(img, code= cv2.COLOR_BGR2GRAY)

In [3]:
# Loading the required haar-cascade xml classifier file 

haar_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

In [4]:
assert not haar_cascade.empty()

In [5]:
help(haar_cascade.detectMultiScale)

Help on built-in function detectMultiScale:

detectMultiScale(...) method of cv2.CascadeClassifier instance
    detectMultiScale(image[, scaleFactor[, minNeighbors[, flags[, minSize[, maxSize]]]]]) -> objects
    .   @brief Detects objects of different sizes in the input image. The detected objects are returned as a list
    .       of rectangles.
    .
    .       @param image Matrix of the type CV_8U containing an image where objects are detected.
    .       @param objects Vector of rectangles where each rectangle contains the detected object, the
    .       rectangles may be partially outside the original image.
    .       @param scaleFactor Parameter specifying how much the image size is reduced at each image scale.
    .       @param minNeighbors Parameter specifying how many neighbors each candidate rectangle should have
    .       to retain it.
    .       @param flags Parameter with the same meaning for an old cascade as in the function
    .       cvHaarDetectObjects. It i

In [6]:
faces_rectangles = haar_cascade.detectMultiScale(
    image= gray_img,
    scaleFactor= 1.1,
    minNeighbors= 5
)

In [7]:
faces_rectangles

array([[ 740,  424,   64,   64],
       [  64,  283,  306,  306],
       [ 638,  462,   58,   58],
       [1016,  320,   81,   81],
       [ 788,  335,  117,  117]])

In [8]:
for (x, y, width, height) in faces_rectangles:
    cv2.rectangle(
        img= img,
        pt1= (x, y),
        pt2= (x + width, y + height),
        color= (0, 0, 255),
        thickness= 2
    )

In [9]:
cv2.imshow('detected_faces', img)
cv2.waitKey(0)

-1

## In real time

In [10]:
cap = cv2.VideoCapture(0)


while cap.isOpened():
    ret, frame = cap.read()

    if not ret:
        break

    gray_frame = cv2.cvtColor(frame, code= cv2.COLOR_BGR2GRAY)

    faces_rect = haar_cascade.detectMultiScale(
        image= gray_frame,
        scaleFactor= 1.1,
        minNeighbors= 5
    )
    for (x, y, w, h) in faces_rect:
        cv2.rectangle(
            img= frame,
            pt1= (x, y),
            pt2= (x + w, y + h),
            color= (255, 0 , 0),
            thickness= 2
        )

    cv2.imshow('Face Detection', frame)
    
    if cv2.waitKey(1) & 0xff == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

## DNN Face Detection

In [22]:
import numpy as np

In [74]:
img = cv2.imread('1.jpg')

# we'll use these to determine the width and height of bounding box from percentages
height, width = img.shape[:2]    

In [75]:
# loading pretrained model
face_net = cv2.dnn.readNetFromCaffe(
    r'E:\Python\Computer Visions\pretrained_models\deploy.prototxt',
    r'E:\Python\Computer Visions\pretrained_models\res10_300x300_ssd_iter_140000.caffemodel'
)

### What Does This Do?
- Loads a Deep Learning model for face detection using OpenCV’s DNN module.
- This model is based on Single Shot MultiBox Detector (SSD) with a ResNet-10 backbone.
- It's trained using Caffe, a deep learning framework.
### What Are These Two Files?
1. `deploy.prototxt`
- Defines the architecture of the neural network (layers, connections, etc.).
- Like a blueprint for the model.
2. `res10_300x300_ssd_iter_140000.caffemodel`
- Contains the pretrained weights (parameters learned during training).
- Used to predict faces in images.

In [76]:
blob = cv2.dnn.blobFromImage(
    img, 
    scalefactor= 1.0,
    size= (300, 300),
    mean= (104.0, 177.0, 123.0)   # mean values of dataset used to train res10_300x300_ssd_iter_140000
)

In [77]:
face_net.setInput(blob)
detections = face_net.forward()

In [78]:
detections.shape

(1, 1, 200, 7)

`200` -> number of detected faces

`7` -> each detected face has 7 values:

1. Batch ID (always `0` since we process one image at a time)
2. Class ID (always `1` because this model only detects faces)
3. Confidence Score (how confident the model is)
4. Bounding Box Coordinates:
    - x_min (Left)
    - y_min (Top)
    - x_max (Right)
    - y_max (Bottom)

Example:
```
[[[[ 0.          1.          0.98       0.23  0.15  0.55  0.75 ]
   [ 0.          1.          0.85       0.65  0.20  0.95  0.80 ]]]]
```
- Face 1
    - Confidence: 98%
    - Bounding Box: (23%, 15%) → (55%, 75%)
- Face 2
    - Confidence: 85%
    - Bounding Box: (65%, 20%) → (95%, 80%)

In [ ]:
for i in range(detections.shape[2]):
    confidence = detections[0, 0, i, 2]

    # taking only those faces with more than 20% confidence 
    if confidence > 0.2:
        box = detections[0, 0, i, 3:7] * np.array([width, height, width, height])
        (x1, y1, x2, y2) = box.astype(int)

        cv2.rectangle(
            img= img,
            pt1= (x1, y1),
            pt2= (x2, y2),
            color= (0, 255, 0),
            thickness= 2
        )

        cv2.putText(
            img= img,
            text= f'Confidence: {confidence:.2%}',
            org= (x1, max(y1 - 10, 0)),
            fontFace= cv2.FONT_HERSHEY_COMPLEX,
            fontScale= 0.5,
            color= (0, 255, 0),
            thickness= 2
        )

cv2.imshow('Face Detection', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

## In real time

In [86]:
cap = cv2.VideoCapture(0)
cap.set(3, 640)    # width
cap.set(4, 480)    # height


while cap.isOpened():
    # reading cap
    ret, frame = cap.read()

    # if no frame found
    if not ret:
        break

    # getting h, w of frame
    height, width = frame.shape[:2]
    
    # preprocessing for DNN
    blob = cv2.dnn.blobFromImage(
        frame, 
        scalefactor= 1.0,
        size= (300, 300),
        mean= (104.0, 177.0, 123.0)
    )

    # getting detections
    face_net.setInput(blob)
    detections = face_net.forward()

    # puttingh bounding boxes and confidence
    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]

        # taking only those faces with more than 70% confidence 
        if confidence > 0.7:
            box = detections[0, 0, i, 3:7] * np.array([width, height, width, height])
            (x1, y1, x2, y2) = box.astype(int)

            cv2.rectangle(
                img= frame,
                pt1= (x1, y1),
                pt2= (x2, y2),
                color= (255, 0, 0),
                thickness= 2
            )

            cv2.putText(
                img= frame,
                text= f'Confidence: {confidence:.2%}',
                org= (x1, max(y1 - 10, 0)),
                fontFace= cv2.FONT_HERSHEY_COMPLEX,
                fontScale= 0.5,
                color= (255, 0, 0),
                thickness= 2
            )

    # displaying
    cv2.imshow('Face Detection', frame)

    if cv2.waitKey(1) & 0xff == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()